**Author:** Elias Rafael Ruiz-Morales

**Institution:** Wellcome Sanger institute

**November, 2023**


---

# Integration of 24hpi and 48hpi datasets

In [1]:
from __future__ import print_function
import torch

import sys, os
data_type = 'float32'
os.environ["THEANO_FLAGS"] = 'device=cuda,floatX=' + data_type + ',force_device=True'
sys.path.insert(1, './')

In [2]:
# Seed for reproducibility
import numpy as np
import pandas as pd
import scanpy as sc
from typing import Tuple
import gc

# scVI imports
import scvi
from scvi.dataset import AnnDatasetFromAnnData
from scvi.inference import UnsupervisedTrainer
from scvi.models.vae import VAE

torch.manual_seed(0)
np.random.seed(0)
sc.settings.verbosity = 0  # verbosity: errors (0), warnings (1), info (2), hints (3)



def MovePlots(plotpattern, subplotdir):
    os.system('mkdir -p '+str(sc.settings.figdir)+'/'+subplotdir)
    os.system('mv '+str(sc.settings.figdir)+'/*'+plotpattern+'** '+str(sc.settings.figdir)+'/'+subplotdir)

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.figdir = '../results/scVI/'
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80)  # low dpi (dots per inch) yields small inline figures

sys.executable

-----
anndata     0.7.6
scanpy      1.7.0
sinfo       0.3.1
-----
PIL                 8.3.0
absl                NA
anndata             0.7.6
anyio               NA
attr                21.2.0
babel               2.9.1
backcall            0.2.0
beta_ufunc          NA
binom_ufunc         NA
brotli              NA
cairo               1.20.1
certifi             2021.05.30
cffi                1.14.4
chardet             4.0.0
colorama            0.4.4
cycler              0.10.0
cython_runtime      NA
dateutil            2.8.1
debugpy             1.3.0
decorator           5.0.9
defusedxml          0.7.1
dunamai             1.5.5
fsspec              2021.06.1
get_version         3.2
google              NA
h5py                3.3.0
idna                2.10
igraph              0.8.3
ipykernel           6.0.0
ipython_genutils    0.2.0
ipywidgets          7.6.3
jedi                0.18.0
jinja2              3.0.1
joblib              1.0.1
json5               NA
jsonschema          3.2.0
jupyter_ser

'/opt/conda/envs/scvi-singularity/bin/python'

In [3]:
def identityTransfer(adata_CellsNewID, adata, field='cell_type'):
    
    '''
    Function to transfer the value in a .obs column, modifying only the cells included in adata_CellsNewID
    
    Parameters:
        adata_CellsNewID: anndata object with cells. These should have in their .obs[field] the new values to be transfer to adata
        adata: anndata object to modify. The value in .obs[field] will be modifies for all the cells in adata_CellsNewID
        field: column name in .obs to operate in.
        
     Return:
        anndata object with the metadata modified
        
    '''
    
    #converting categorical into strings to introduce new categories
    adata.obs[field]=adata.obs[field].astype("string")


    #assigning the new categories to the cells in adata
    for cell in adata_CellsNewID.obs.index:
        adata.obs[field][cell] = adata_CellsNewID.obs[field][cell]

    #Returning strings into categorical 
    adata.obs[field]=adata.obs[field].astype("category")
    
    return(adata)

### Loading non-normalized data

In [4]:
adata_24h = sc.read('../../24h/results/scVI/rna8_scVIintegrated_latent30_All_20230707.h5ad')

adata_48h = sc.read('../../48h/results/scVI/rna8_scVIintegrated_latent30_All_20230707_48h.h5ad')

In [5]:
del(adata_24h.obs["cell_type_2022"])
del(adata_24h.uns["cell_type_2022_colors"])

In [6]:
adata_24h

AnnData object with n_obs × n_vars = 113028 × 36601
    obs: 'sample', 'stage', 'hpi', 'infection', 'percent_mito', 'n_counts', 'sample_barcode', 'assignment_SoC', 'donor_id', 'scrublet_score', 'scrublet_cluster_score', 'zscore', 'bh_pval', 'bonf_pval', 'S_score', 'G2M_score', 'phase', 'n_genes_by_counts', 'total_counts', 'total_counts_hs', 'pct_counts_hs', 'total_counts_tg', 'pct_counts_tg', 'Tg_infected', 'n_genes', '_scvi_batch', '_scvi_labels', '_scvi_local_l_mean', '_scvi_local_l_var', 'leiden_scvi', 'celltype_predictions', 'probabilities', 'scrublet_doublet', 'cell_type', 'souporcell_MFgenotype', 'MFgenotype', 'cell_type_broad', 'umap_density_Tg_infected', 'stage_perInfection', 'celltype-Stage', 'Tg_intracellular', 'celltype-Intracellular', 'Dev_Stage'
    var: 'gene_ids', 'feature_types', 'mean-0', 'std-0', 'mean-1', 'std-1', 'mean-2', 'std-2', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'Dev_Stage_colors', 'MFgenotype_colors', '_sc

In [7]:
adata_48h

AnnData object with n_obs × n_vars = 45950 × 36601
    obs: 'sample', 'stage', 'hpi', 'infection', 'percent_mito', 'n_counts', 'sample_barcode', 'assignment_SoC', 'donor_id', 'scrublet_score', 'scrublet_cluster_score', 'zscore', 'bh_pval', 'bonf_pval', 'S_score', 'G2M_score', 'phase', 'n_genes_by_counts', 'total_counts', 'total_counts_hs', 'pct_counts_hs', 'total_counts_tg', 'pct_counts_tg', 'Tg_infected', 'n_genes', '_scvi_batch', '_scvi_labels', '_scvi_local_l_mean', '_scvi_local_l_var', 'leiden_scvi', 'celltype_predictions', 'probabilities', 'scrublet_doublet', 'cell_type', 'souporcell_MFgenotype', 'MFgenotype', 'cell_type_broad', 'stage_perInfection', 'celltype-Stage', 'Tg_intracellular', 'celltype-Intracellular', 'Dev_Stage'
    var: 'gene_ids', 'feature_types', 'mean-0', 'std-0', 'mean-1', 'std-1', 'mean-2', 'std-2', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'Dev_Stage_colors', 'MFgenotype_colors', '_scvi', 'cell_type_broad_colors'

## Concatenate datasets

In [8]:
adata = adata_24h.concatenate(adata_48h, index_unique=None)

/opt/conda/envs/scvi-singularity/lib/python3.8/site-packages/anndata/_core/merge.py:895: UserWarning: Only some AnnData objects have `.raw` attribute, not concatenating `.raw` attributes.
  warn(


In [9]:
adata

AnnData object with n_obs × n_vars = 158978 × 36601
    obs: 'sample', 'stage', 'hpi', 'infection', 'percent_mito', 'n_counts', 'sample_barcode', 'assignment_SoC', 'donor_id', 'scrublet_score', 'scrublet_cluster_score', 'zscore', 'bh_pval', 'bonf_pval', 'S_score', 'G2M_score', 'phase', 'n_genes_by_counts', 'total_counts', 'total_counts_hs', 'pct_counts_hs', 'total_counts_tg', 'pct_counts_tg', 'Tg_infected', 'n_genes', '_scvi_batch', '_scvi_labels', '_scvi_local_l_mean', '_scvi_local_l_var', 'leiden_scvi', 'celltype_predictions', 'probabilities', 'scrublet_doublet', 'cell_type', 'souporcell_MFgenotype', 'MFgenotype', 'cell_type_broad', 'umap_density_Tg_infected', 'stage_perInfection', 'celltype-Stage', 'Tg_intracellular', 'celltype-Intracellular', 'Dev_Stage', 'batch'
    var: 'gene_ids', 'feature_types', 'mean-0-0', 'std-0-0', 'mean-1-0', 'std-1-0', 'mean-2-0', 'std-2-0', 'highly_variable-0', 'highly_variable_rank-0', 'means-0', 'variances-0', 'variances_norm-0', 'mean-0-1', 'std-0-1',

In [10]:
del(adata_24h,adata_48h)

In [11]:
gc.collect()

396

### Compute the scVI latent space

Based on the scVI documentation.

In [12]:
# do some basic preprocessing
#adata.layers["raw_counts"] = adata.X.copy() # preserve counts
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
#adata.raw = adata.copy()

normalizing counts per cell
    finished (0:00:02)


In [13]:
np.unique(adata.obs['donor_id'])

array(['Hrv124', 'Hrv135', 'Hrv136', 'Hrv168', 'Hrv232', 'Hrv236',
       'scDonor_Tg1', 'scDonor_Tg2', 'scDonor_Tg3', 'scDonor_Tg4'],
      dtype=object)

In [14]:
adata.obs['sample']

Pla_HDBR13007974_AAACCCAAGCGTTGTT    Pla_HDBR13007974
Pla_HDBR13007974_AAACCCAAGTAGTCAA    Pla_HDBR13007974
Pla_HDBR13007974_AAACCCACAATGAACA    Pla_HDBR13007974
Pla_HDBR13007974_AAACCCACAGAGAGGG    Pla_HDBR13007974
Pla_HDBR13007974_AAACCCACAGTAGAAT    Pla_HDBR13007974
                                           ...       
Pla_HDBR13661576_TTTGTTGAGAGTACCG    Pla_HDBR13661576
Pla_HDBR13661576_TTTGTTGAGCATATGA    Pla_HDBR13661576
Pla_HDBR13661576_TTTGTTGCATCCCGTT    Pla_HDBR13661576
Pla_HDBR13661576_TTTGTTGTCAGCTGAT    Pla_HDBR13661576
Pla_HDBR13661576_TTTGTTGTCTGGTTGA    Pla_HDBR13661576
Name: sample, Length: 158978, dtype: object

In [15]:
adata.obs

,sample,stage,hpi,infection,percent_mito,n_counts,sample_barcode,assignment_SoC,donor_id,scrublet_score,...,souporcell_MFgenotype,MFgenotype,cell_type_broad,umap_density_Tg_infected,stage_perInfection,celltype-Stage,Tg_intracellular,celltype-Intracellular,Dev_Stage,batch
Pla_HDBR13007974_AAACCCAAGCGTTGTT,Pla_HDBR13007974,UI_24h,24h,UI,0.019590,6432.0,Pla_HDBR13007974_AAACCCAAGCGTTGTT,1,scDonor_Tg2,0.020057,...,0,Maternal,F,0.365844,UI_Tg_24h,F-UI_Tg_24h,UI,F-UI_Tg_24h,12pcw,0
Pla_HDBR13007974_AAACCCAAGTAGTCAA,Pla_HDBR13007974,UI_24h,24h,UI,0.045489,49221.0,Pla_HDBR13007974_AAACCCAAGTAGTCAA,0,scDonor_Tg1,0.092913,...,3,Fetal,VCT,0.312446,UI_Tg_24h,VCT_fusing-UI_Tg_24h,UI,VCT_fusing-UI_Tg_24h,CS22,0
Pla_HDBR13007974_AAACCCACAATGAACA,Pla_HDBR13007974,UI_24h,24h,UI,0.045332,9243.0,Pla_HDBR13007974_AAACCCACAATGAACA,1,scDonor_Tg2,0.085202,...,0,Maternal,HBC,0.903018,UI_Tg_24h,HBC-UI_Tg_24h,UI,HBC-UI_Tg_24h,12pcw,0
Pla_HDBR13007974_AAACCCACAGAGAGGG,Pla_HDBR13007974,UI_24h,24h,UI,0.031214,7753.0,Pla_HDBR13007974_AAACCCACAGAGAGGG,1,scDonor_Tg2,0.099262,...,0,Maternal,HBC,0.633648,UI_Tg_24h,HBC-UI_Tg_24h,UI,HBC-UI_Tg_24h,12pcw,0
Pla_HDBR13007974_AAACCCACAGTAGAAT,Pla_HDBR13007974,UI_24h,24h,UI,0.043799,14361.0,Pla_HDBR13007974_AAACCCACAGTAGAAT,0,scDonor_Tg1,0.059619,...,3,Fetal,HBC,0.794102,UI_Tg_24h,HBC-UI_Tg_24h,UI,HBC-UI_Tg_24h,CS22,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pla_HDBR13661576_TTTGTTGAGAGTACCG,Pla_HDBR13661576,Lm_48h,48h,Lm,0.071074,50722.0,Pla_HDBR13661576_TTTGTTGAGAGTACCG,not_pooled,Hrv236,0.091183,...,1,Fetal,VCT,NaN,Lm_48h,VCT-Lm_48h,nan,VCT-Lm_48h,14pcw,1
Pla_HDBR13661576_TTTGTTGAGCATATGA,Pla_HDBR13661576,Lm_48h,48h,Lm,0.086808,16358.0,Pla_HDBR13661576_TTTGTTGAGCATATGA,not_pooled,Hrv236,0.085631,...,1,Fetal,HBC,NaN,Lm_48h,HBC-Lm_48h,nan,HBC-Lm_48h,14pcw,1
Pla_HDBR13661576_TTTGTTGCATCCCGTT,Pla_HDBR13661576,Lm_48h,48h,Lm,0.018906,12800.0,Pla_HDBR13661576_TTTGTTGCATCCCGTT,not_pooled,Hrv236,0.170507,...,1,Fetal,PV,NaN,Lm_48h,PV-Lm_48h,nan,PV-Lm_48h,14pcw,1
Pla_HDBR13661576_TTTGTTGTCAGCTGAT,Pla_HDBR13661576,Lm_48h,48h,Lm,0.049094,21082.0,Pla_HDBR13661576_TTTGTTGTCAGCTGAT,not_pooled,Hrv236,0.059382,...,1,Fetal,Endo_f,NaN,Lm_48h,Endo_f-Lm_48h,nan,Endo_f-Lm_48h,14pcw,1


In [16]:
scvi.data.setup_anndata(
    adata,
    layer="raw_counts",
    batch_key='donor_id', #samples as a batch
    #donor as a covariate of the cells
    categorical_covariate_keys=["sample"] 
    #categorical_covariate_keys=['donor_souporcell',], #
    #continuous_covariate_keys=[""]
)

INFO     Using batches from adata.obs["donor_id"]                                            


Using batches from adata.obs["donor_id"]


INFO     No label_key inputted, assuming all cells have same label                           


No label_key inputted, assuming all cells have same label


INFO     Using data from adata.layers["raw_counts"]                                          


...

Computing library size prior per batch


INFO     Successfully registered anndata object containing 158978 cells, 36601 vars, 10      
         batches, 1 labels, and 0 proteins. Also registered 1 extra categorical covariates   
         and 0 extra continuous covariates.                                                  


Successfully registered anndata object containing 158978 cells, 36601 vars, 10 batches, 1 labels, and 0 proteins. Also registered 1 extra categorical covariates and 0 extra continuous covariates.


INFO     Please do not further modify adata until model is trained.                          


Please do not further modify adata until model is trained.


In [17]:
#---- check #layers

In [18]:
models = {}

# let's try a few values
n_latent_values = [20, 30]

for n_latent_value in n_latent_values:
    print('n_latent_value', n_latent_value)
    models[n_latent_value] = scvi.model.SCVI(adata, n_latent = n_latent_value)

n_latent_value 20
n_latent_value 30


In [19]:
models[20]

SCVI Model with the following params: 
n_hidden: 128, n_latent: 20, n_layers: 1, dropout_rate: 0.1, dispersion: gene, 
gene_likelihood: zinb, latent_distribution: normal
Training status: Not Trained

To print summary of associated AnnData, use: scvi.data.view_anndata_setup(model.adata)

In [20]:
latent_representations = {}

for n_latent_value in n_latent_values:
    print('training model for n_latent_value:', n_latent_value)
    models[n_latent_value].train()
    
    latent_representations[n_latent_value] = models[n_latent_value].get_latent_representation()
    
    adata.obsm["X_scVI_n_latent_" + str(n_latent_value)] = latent_representations[n_latent_value]
    
    curr_df = pd.DataFrame(adata.obsm["X_scVI_n_latent_" + str(n_latent_value)])
    curr_df.to_csv('../results/20230707_obsm_with_scVI_latent_representation_n_' + str(n_latent_value) + '_allSamplesAllTimePoints.csv')

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


training model for n_latent_value: 20
Epoch 50/50: 100%|███████████████████████████████████████████| 50/50 [25:40<00:00, 30.81s/it, loss=1.27e+04, v_num=1]


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


training model for n_latent_value: 30
Epoch 50/50: 100%|███████████████████████████████████████████| 50/50 [25:28<00:00, 30.56s/it, loss=1.27e+04, v_num=1]


In [21]:
adata.X= adata.layers["raw_counts"].copy()

In [22]:
adata.write('../results/rna_scVIintegrated_AllData.h5ad')

... storing 'sample' as categorical
... storing 'stage' as categorical
... storing 'hpi' as categorical
... storing 'assignment_SoC' as categorical
... storing 'donor_id' as categorical
... storing 'celltype_predictions' as categorical
... storing 'cell_type' as categorical
... storing 'souporcell_MFgenotype' as categorical
... storing 'stage_perInfection' as categorical
... storing 'celltype-Stage' as categorical
... storing 'celltype-Intracellular' as categorical
... storing 'Dev_Stage' as categorical


In [23]:
adata.var["feature_types"].value_counts()


Gene Expression    36601
Name: feature_types, dtype: int64

In [24]:
adata

AnnData object with n_obs × n_vars = 158978 × 36601
    obs: 'sample', 'stage', 'hpi', 'infection', 'percent_mito', 'n_counts', 'sample_barcode', 'assignment_SoC', 'donor_id', 'scrublet_score', 'scrublet_cluster_score', 'zscore', 'bh_pval', 'bonf_pval', 'S_score', 'G2M_score', 'phase', 'n_genes_by_counts', 'total_counts', 'total_counts_hs', 'pct_counts_hs', 'total_counts_tg', 'pct_counts_tg', 'Tg_infected', 'n_genes', '_scvi_batch', '_scvi_labels', '_scvi_local_l_mean', '_scvi_local_l_var', 'leiden_scvi', 'celltype_predictions', 'probabilities', 'scrublet_doublet', 'cell_type', 'souporcell_MFgenotype', 'MFgenotype', 'cell_type_broad', 'umap_density_Tg_infected', 'stage_perInfection', 'celltype-Stage', 'Tg_intracellular', 'celltype-Intracellular', 'Dev_Stage', 'batch'
    var: 'gene_ids', 'feature_types', 'mean-0-0', 'std-0-0', 'mean-1-0', 'std-1-0', 'mean-2-0', 'std-2-0', 'highly_variable-0', 'highly_variable_rank-0', 'means-0', 'variances-0', 'variances_norm-0', 'mean-0-1', 'std-0-1',